In [5]:
! ls /usr/local/

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [6]:
! nvcc --version # nvcc compiler version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [29]:
%%writefile MatrixMultiplication.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>

/* Variables residing in shared memory can be accessed at very high speed on a higly parallel manner;
all threads in the same block have access to shared memory */

#define BlockSize 32

// Kernel Function
__global__ void MatrixMultiplication(float *A, float *B, float *C, int N, int K, int M){
  __shared__ float As[BlockSize][BlockSize]; // shared memory for matrix A
  __shared__ float Bs[BlockSize][BlockSize]; // shared memory for matrix B

  int bx = blockIdx.x; // blocks indices
  int by = blockIdx.y;

  int tx = threadIdx.x; // threads indices
  int ty = threadIdx.y;

  int ABegin = K * BlockSize * by;
  int AEnd = ABegin + K - 1;
  int AStep = BlockSize;

  int BBegin = BlockSize * bx;
  int BStep = BlockSize * M;

  float Sum = 0.0f;

  for(int a = ABegin, b = BBegin; a <= AEnd; a += AStep, b += BStep){
    As[ty][tx] = A[a + K * ty + tx];
    Bs[ty][tx] = B[b + M * ty + tx];

    __syncthreads();

    #pragma unroll
    for(int i = 0; i < BlockSize; i++){
      Sum += As[ty][i] * Bs[i][tx];
    }

    __syncthreads();
  }

  int c = M * BlockSize * by + BlockSize * bx;
  C[c + M * ty + tx] = Sum;
}

// Main code executed by the host
int main(void){

  int N = 12; // rows matrix A
  int K = 14; // columns matrix A; rows matrix B
  int M = 16; // columns matrix B

  float *Ah, *Bh, *Ch; // host matrix pointers
  Ah = (float *)malloc(N*K*sizeof(float)); // allocate host memory
  Bh = (float *)malloc(K*M*sizeof(float));
  Ch = (float *)malloc(N*M*sizeof(float));

  for(int i = 0; i < N; i++){ // initialize elements matrix A
    for(int j = 0; j < K; j++){
      Ah[i*K+j] = i + 1.0;
    }
  }

  for(int i = 0; i < K; i++){ // initialize elements matrix B
    for(int j = 0; j < M; j++){
      Bh[i*M+j] = (i + 1.0)*2;
    }
  }

  printf("\n Matrix A. \n");
  for(int i = 0; i < N; i++){
    for(int j = 0; j < K; j++){
      printf("%.2lf ", Ah[i*K+j]);
    }
    printf("\n");
  }

  printf("\n Matrix B. \n");
  for(int i = 0; i < K; i++){
    for(int j = 0; j < M; j++){
      printf("%.2lf ", Bh[i*M+j]);
    }
    printf("\n");
  }

  float *Ad, *Bd, *Cd;
  cudaMalloc((void **)&Ad, sizeof(float) * N * K); // allocate GPU memory
  cudaMalloc((void **)&Bd, sizeof(float) * K * M);
  cudaMalloc((void **)&Cd, sizeof(float) * N * M);

  cudaMemcpy(Ad, Ah, sizeof(float) * N * K, cudaMemcpyHostToDevice); // copy data from host to device (matrix A)
  cudaMemcpy(Bd, Bh, sizeof(float) * K * M, cudaMemcpyHostToDevice); // copy data from host to device (matrix B)

  dim3 blockSize(BlockSize, BlockSize); // threads per block
  dim3 GridSize((M - 1) / blockSize.x + 1, (N - 1) / blockSize.y + 1); // no. of blocks

  MatrixMultiplication<<<GridSize, blockSize>>>(Ad, Bd, Cd, N, K, M); // kernel function calling

  cudaMemcpy(Ch, Cd, sizeof(float) * N * M, cudaMemcpyDeviceToHost); // copy data from device back to host (matrix C)

  printf("\n Matrix C. \n");
  for(int i = 0; i < N; i++){
    for(int j = 0; j < M; j++){
      printf("%.2lf ", Ch[i*M+j]);
    }
    printf("\n");
  }

  free(Ah); // free host memory
  free(Bh);
  free(Ch);

  cudaFree(Ad); // free GPU memory
  cudaFree(Bd);
  cudaFree(Cd);

  return(0);
}

Overwriting MatrixMultiplication.cu


In [30]:
! nvcc MatrixMultiplication.cu -o test

In [31]:
! ./test


 Matrix A. 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 4.00 
5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 5.00 
6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 
7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 7.00 
8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 8.00 
9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 9.00 
10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 10.00 
11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 11.00 
12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 12.00 

 Matrix B. 
2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 